Hello everyone! This is an example of building a simple language recognition model for a Ukrainian Open Speech To Text Dataset

Based on this example: https://www.tensorflow.org/tutorials/audio/simple_audio

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import re
from sklearn.model_selection import train_test_split
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf

from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers
from tensorflow.keras import models
from IPython import display
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from collections import Counter
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
data=pd.read_csv('../input/ukrainian-ostotextdataset42-dataframe/Ukrainian_Open_Speech_To_Text Dataset.csv')

data['number_of_words'] = data['text'].apply(lambda x: len(str(x).split()))
data['words'] = data['text'].apply(lambda x: str(x).split())
print(data)


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
print(data.shape)

print(data.loc[data['number_of_words'] == 1])


In [ ]:
datasets=data['dataset'].unique()
data=data.dropna()#.fillna(value='', inplace=True)
#data=data.loc[data['dataset'] == 'VR/']

#data=data.loc[data['dataset'] != '1TVUKRAINIAN/']
#data=data.loc[data['dataset'] != 'GROSHI/']
#data=data.loc[data['dataset'] != 'MON/']

data=data.loc[data['number_of_words'] == 1]
data['count'] = data.groupby('text')['text'].transform('count')

data=data.loc[data['count'] > 10]
data=data[['path','text']]
data['text'] = data['text'].str.replace(',','-')
print('kaggle datasets:',datasets)
commands=data['text'].unique()
print('commands:',len(commands))


In [ ]:
#data=data.dropna()#.fillna(value='', inplace=True)

In [ ]:
print(len(data))
val_size=0.2#0.2
test_size=0.1
#val_size=round(val_size*len(data))
#test_size=round(test_size*len(data))

from sklearn.model_selection import train_test_split

train_val_files_pd,test_files_pd,_,__= train_test_split(data,data['text'],test_size=test_size,stratify=data['text'], random_state=10)

train_files_pd,val_files_pd,_,__= train_test_split(train_val_files_pd,train_val_files_pd['text'],test_size=val_size,stratify=train_val_files_pd['text'], random_state=11)
#train_files_pd = data[:len(data)-val_size-test_size]
#val_files_pd = data[len(data)-val_size-test_size: len(data)-test_size]
#test_files_pd = data[len(data)-test_size:]

train_files=train_files_pd.to_numpy()
val_files=val_files_pd.to_numpy()
test_files=test_files_pd.to_numpy()

print('Training set size', len(train_files))
print('Validation set size', len(val_files))
print('Test set size', len(test_files))

In [ ]:
def get_label(file_path):
  parts = tf.strings.split(file_path, os.path.sep)

  # Note: You'll use indexing here instead of tuple unpacking to enable this 
  # to work in a TensorFlow graph.
  return parts[-2]

In [ ]:
def decode_audio(audio_binary):
  audio, _ = tf.audio.decode_wav(audio_binary)
  return tf.squeeze(audio, axis=-1)

In [ ]:
def get_waveform_and_label(file_path):
    label = file_path[1]
    audio_binary = tf.io.read_file(file_path[0])
    print(audio_binary)
    waveform = decode_audio(audio_binary)
    return waveform, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
files_ds = tf.data.Dataset.from_tensor_slices(train_files)
print(files_ds)
waveform_ds = files_ds.map(get_waveform_and_label, num_parallel_calls=AUTOTUNE)
print(files_ds)


In [ ]:
rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(10, 12))
for i, (audio, label) in enumerate(waveform_ds.take(n)):
  r = i // cols
  c = i % cols
  ax = axes[r][c]
  ax.plot(audio.numpy())
  ax.set_yticks(np.arange(-1.2, 1.2, 0.2))
  label = label.numpy().decode('utf-8')
  ax.set_title(label)

plt.show()

In [ ]:
def get_spectrogram(waveform):
  # Padding for files with less than 16000 samples
  #if waveform.get_shape().as_list()[0]:
  print (([50000] -tf.shape(waveform))<0)
  if ([50000] -tf.shape(waveform))<0:
    waveform=tf.slice(waveform, [0], [50000])
  zero_padding = tf.zeros([50000] - tf.shape(waveform), dtype=tf.float32)
  #print(waveform.get_shape().as_list()[0])  
  #print(tf.slice(waveform, [0], [100000], name=None


  # Concatenate audio with padding so that all audio clips will be of the 
  # same length
  waveform = tf.cast(waveform, tf.float32)
  equal_length = tf.concat([waveform, zero_padding], 0)
  spectrogram = tf.signal.stft(
      equal_length, frame_length=255, frame_step=128)

  spectrogram = tf.abs(spectrogram)

  return spectrogram

In [ ]:
for waveform, label in waveform_ds.take(1):
  label = label.numpy().decode('utf-8')
  spectrogram = get_spectrogram(waveform)

print('Label:', label)
print('Waveform shape:', waveform.shape)
print('Spectrogram shape:', spectrogram.shape)
print('Audio playback')
display.display(display.Audio(waveform, rate=16000))

In [ ]:
def plot_spectrogram(spectrogram, ax):
  # Convert to frequencies to log scale and transpose so that the time is
  # represented in the x-axis (columns).
  log_spec = np.log(spectrogram.T)
  height = log_spec.shape[0]
  width = log_spec.shape[1]
  X = np.linspace(0, np.size(spectrogram), num=width, dtype=int)
  Y = range(height)
  ax.pcolormesh(X, Y, log_spec)


fig, axes = plt.subplots(2, figsize=(12, 8))
timescale = np.arange(waveform.shape[0])
axes[0].plot(timescale, waveform.numpy())
axes[0].set_title('Waveform')
axes[0].set_xlim([0, 16000])
plot_spectrogram(spectrogram.numpy(), axes[1])
axes[1].set_title('Spectrogram')
plt.show()

In [ ]:
def get_spectrogram_and_label_id(audio, label):
  spectrogram = get_spectrogram(audio)
  spectrogram = tf.expand_dims(spectrogram, -1)
  label_id = tf.argmax(label == commands)
  return spectrogram, label_id

In [ ]:
spectrogram_ds = waveform_ds.map(
    get_spectrogram_and_label_id, num_parallel_calls=AUTOTUNE)

In [ ]:
rows = 3
cols = 3
n = rows*cols
fig, axes = plt.subplots(rows, cols, figsize=(10, 10))
for i, (spectrogram, label_id) in enumerate(spectrogram_ds.take(n)):
  r = i // cols
  c = i % cols
  ax = axes[r][c]
  plot_spectrogram(np.squeeze(spectrogram.numpy()), ax)
  ax.set_title(commands[label_id.numpy()])
  ax.axis('off')

plt.show()

In [ ]:
def preprocess_dataset(files):
  files_ds = tf.data.Dataset.from_tensor_slices(files)
  output_ds = files_ds.map(get_waveform_and_label, num_parallel_calls=AUTOTUNE)
  output_ds = output_ds.map(
      get_spectrogram_and_label_id,  num_parallel_calls=AUTOTUNE)
  return output_ds

In [ ]:
train_ds = spectrogram_ds
val_ds = preprocess_dataset(val_files)
test_ds = preprocess_dataset(test_files)

In [ ]:
batch_size = 128
train_ds = train_ds.batch(batch_size)
val_ds = val_ds.batch(batch_size)

In [ ]:
for spectrogram, _ in spectrogram_ds.take(1):
  input_shape = spectrogram.shape
print('Input shape:', input_shape)
num_labels = len(commands)
print('num_labels:', num_labels)
norm_layer = preprocessing.Normalization()
#norm_layer.adapt(spectrogram_ds.map(lambda x, _: x))

model = models.Sequential([
    layers.Input(shape=input_shape),
    preprocessing.Resizing(96, 96), 
    norm_layer,
    layers.Conv2D(512, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Conv2D(256, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_labels),
])

model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

In [ ]:
EPOCHS = 30
history = model.fit(
    train_ds, 
    #steps_per_epoch =20,
    validation_data=val_ds,  
    epochs=EPOCHS,
    #callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
)

In [ ]:
metrics = history.history
plt.plot(history.epoch, metrics['loss'], metrics['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()

In [ ]:
test_audio = []
test_labels = []

for audio, label in test_ds:
  test_audio.append(audio.numpy())
  test_labels.append(label.numpy())

test_audio = np.array(test_audio)
test_labels = np.array(test_labels)

In [ ]:
y_pred = np.argmax(model.predict(test_audio), axis=1)
y_true = test_labels

test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test set accuracy: {test_acc:.0%}')